In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.special import j0  # 第1種0次ベッセル関数
from scipy.optimize import curve_fit
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set matplotlib to use default settings
plt.rcParams.update(plt.rcParamsDefault)

print("Library import completed!")

## 1. 微動観測の基礎

### 1.1 微動とは？
微動（microtremor）は地盤に常時存在する微小な振動です。主な発生源は：
- 海洋波浪（脈動：0.1-1 Hz）
- 交通振動（1-20 Hz）
- 風による構造物の振動
- 人間活動による振動

### 1.2 地震波の種類

地震波は大きく**実体波**（body waves）と**表面波**（surface waves）に分類されます。

#### 実体波
- **P波（Primary wave）**：縦波、速度 $v_p = \sqrt{\frac{\lambda + 2\mu}{\rho}}$
- **S波（Secondary wave）**：横波、速度 $v_s = \sqrt{\frac{\mu}{\rho}}$

#### 表面波
- **レイリー波（Rayleigh wave）**：鉛直面内での楕円運動、微動の主成分（約70%）、速度：約$0.92v_s$（ポアソン比0.25の場合）
- **ラブ波（Love wave）**：水平面内での振動

### 1.3 なぜ微動を使うのか？
- 人工的な振源が不要
- 低周波数成分が豊富（深部構造の探査に有利）
- 都市部でも観測可能

In [ ]:
# 位相速度と群速度の違いを可視化
def visualize_phase_group_velocity():
    """
    2つの正弦波の重ね合わせによる位相速度と群速度の違いを示す
    u(x,t) = sin(k₁x - ω₁t) + sin(k₂x - ω₂t)
           = 2cos(Δk·x - Δω·t)sin(kx - ωt)
    """
    
    # パラメータ設定
    k1, k2 = 1.0, 1.2  # 波数
    omega1, omega2 = 2.0, 2.4  # 角周波数
    k = (k1 + k2) / 2
    omega = (omega1 + omega2) / 2
    dk = (k2 - k1) / 2
    domega = (omega2 - omega1) / 2
    
    # 位相速度と群速度
    c_phase = omega / k  # 位相速度
    c_group = domega / dk  # 群速度
    
    # 時間と空間の配列
    x = np.linspace(0, 50, 1000)
    
    fig, axes = plt.subplots(3, 1, figsize=(12, 10))
    
    for idx, t_val in enumerate([0, 1, 2]):
        ax = axes[idx]
        
        # 2つの波の重ね合わせ
        wave1 = np.sin(k1 * x - omega1 * t_val)
        wave2 = np.sin(k2 * x - omega2 * t_val)
        combined = wave1 + wave2
        
        # 包絡線（群速度で移動）
        envelope = 2 * np.cos(dk * x - domega * t_val)
        
        # 搬送波（位相速度で移動）
        carrier = np.sin(k * x - omega * t_val)
        
        # プロット
        ax.plot(x, combined, 'b-', alpha=0.7, label='Combined wave')
        ax.plot(x, envelope, 'r-', linewidth=2, label='Envelope (Group velocity)')
        ax.plot(x, -envelope, 'r-', linewidth=2)
        ax.fill_between(x, envelope, -envelope, alpha=0.2, color='red')
        
        # 位相の追跡点
        phase_pos = c_phase * t_val
        group_pos = c_group * t_val
        
        # 位相速度と群速度の位置を表示
        if phase_pos < 50:
            ax.axvline(phase_pos, color='green', linestyle='--', 
                      label=f'Phase velocity = {c_phase:.2f}' if idx == 0 else '')
        if group_pos < 50:
            ax.axvline(group_pos, color='orange', linestyle='--', 
                      label=f'Group velocity = {c_group:.2f}' if idx == 0 else '')
        
        ax.set_ylabel('Amplitude')
        ax.set_title(f't = {t_val} s')
        ax.grid(True, alpha=0.3)
        ax.set_xlim(0, 50)
        ax.set_ylim(-2.5, 2.5)
        
        if idx == 0:
            ax.legend()
        if idx == 2:
            ax.set_xlabel('Distance (m)')
    
    plt.tight_layout()
    plt.suptitle('Phase Velocity vs Group Velocity: $u(x,t) = 2\\cos(\\Delta k \\cdot x - \\Delta\\omega \\cdot t)\\sin(kx - \\omega t)$', 
                 y=1.02, fontsize=14)
    plt.show()

# デモンストレーションの実行
visualize_phase_group_velocity()

## 2. 位相速度と群速度

### 2.1 位相速度（Phase velocity）
位相速度$c$は、単一周波数の波の位相が伝播する速度：

$$c = \frac{\omega}{k} = f\lambda$$

### 2.2 群速度（Group velocity）
群速度$U$は、波束（エネルギー）が伝播する速度：

$$U = \frac{d\omega}{dk} = c - \lambda\frac{dc}{d\lambda}$$

### 2.3 分散性媒質での波の伝播
媒質が不均質な場合、位相速度は周波数に依存し（分散性）、これにより地下構造の推定が可能となる。

In [ ]:
# 模擬微動データの生成
def generate_synthetic_microtremor(duration=300, fs=100, frequencies=[0.5, 1.0, 2.0, 5.0], 
                                  amplitudes=[1.0, 0.5, 0.3, 0.1]):
    """
    合成微動データを生成する関数
    
    Parameters:
    - duration: 記録時間 (秒)
    - fs: サンプリング周波数 (Hz)
    - frequencies: 含まれる周波数成分 (Hz)
    - amplitudes: 各周波数成分の振幅
    """
    t = np.arange(0, duration, 1/fs)
    data = np.zeros(len(t))
    
    # 複数の正弦波を重ね合わせて微動を模擬
    for f, a in zip(frequencies, amplitudes):
        phase = np.random.rand() * 2 * np.pi
        data += a * np.sin(2 * np.pi * f * t + phase)
    
    # ノイズを追加
    data += 0.1 * np.random.randn(len(t))
    
    return t, data

# データ生成と可視化
t, microtremor = generate_synthetic_microtremor()

# 時系列データの表示
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(t[:1000], microtremor[:1000])
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title('Microtremor Time Series (First 10 seconds)')
plt.grid(True)

# スペクトルの計算と表示
plt.subplot(1, 2, 2)
f, psd = signal.welch(microtremor, fs=100, nperseg=1024)
plt.semilogy(f, psd)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power Spectral Density')
plt.title('Power Spectral Density')
plt.grid(True)
plt.xlim(0, 10)

plt.tight_layout()
plt.show()

## 2. アレイ観測の概念

微動アレイ観測では、複数の地震計を配置して同時観測を行います。
一般的な配置：
- 円形アレイ（中心＋円周上）
- 三角形アレイ
- L字型アレイ

ここでは、円形アレイを例に解析を進めます。

In [ ]:
# 円形アレイの配置を可視化
def plot_array_configuration(radius=50, n_stations=4):
    """円形アレイの配置を描画"""
    fig, ax = plt.subplots(figsize=(6, 6))
    
    # 中心点
    ax.scatter(0, 0, s=200, c='red', marker='s', label='Center')
    ax.text(0, 5, 'C', ha='center', fontsize=12)
    
    # 円周上の観測点
    angles = np.linspace(0, 2*np.pi, n_stations, endpoint=False)
    for i, angle in enumerate(angles):
        x = radius * np.cos(angle)
        y = radius * np.sin(angle)
        ax.scatter(x, y, s=200, c='blue', marker='^')
        ax.text(x, y+5, f'S{i+1}', ha='center', fontsize=12)
    
    # 円を描画
    circle = plt.Circle((0, 0), radius, fill=False, linestyle='--', color='gray')
    ax.add_patch(circle)
    
    ax.set_xlim(-radius*1.3, radius*1.3)
    ax.set_ylim(-radius*1.3, radius*1.3)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_title(f'Circular Array Configuration (r={radius}m)')
    ax.legend()
    
    return fig

# アレイ配置の表示
plot_array_configuration(radius=50, n_stations=4)
plt.show()

# 複数サイズのアレイ
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
radii = [10, 50, 200]
for ax, r in zip(axes, radii):
    plt.sca(ax)
    plot_array_configuration(radius=r, n_stations=4)
    ax.set_title(f'Array Size: {r}m\nTarget Frequency: {100/r:.1f}-{500/r:.1f} Hz')
plt.tight_layout()
plt.show()

In [ ]:
# SPAC法の原理を可視化
def visualize_spac_principle():
    """空間相関と位相速度の関係を可視化"""
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. 異なる観測点間距離での波形の相関
    ax = axes[0, 0]
    t = np.linspace(0, 2, 500)
    wavelength = 1.0
    
    for i, dist_ratio in enumerate([0, 0.25, 0.5, 1.0]):
        distance = dist_ratio * wavelength
        phase_diff = 2 * np.pi * distance / wavelength
        
        wave1 = np.sin(2 * np.pi * t)
        wave2 = np.sin(2 * np.pi * t - phase_diff)
        
        offset = i * 3
        ax.plot(t, wave1 + offset, 'b-', alpha=0.7)
        ax.plot(t, wave2 + offset, 'r-', alpha=0.7)
        ax.text(2.1, offset, f'$r/\\lambda$ = {dist_ratio:.2f}', fontsize=10)
        
        # 相関係数
        corr = np.cos(phase_diff)
        ax.text(2.5, offset, f'$\\rho$ = {corr:.2f}', fontsize=10, color='green')
    
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Amplitude')
    ax.set_title('Waveform Correlation at Different Distances')
    ax.grid(True, alpha=0.3)
    
    # 2. ベッセル関数の形状
    ax = axes[0, 1]
    x = np.linspace(0, 15, 1000)
    y = j0(x)
    ax.plot(x, y, 'b-', linewidth=2)
    ax.axhline(0, color='k', linewidth=0.5)
    ax.axvline(2.4048, color='r', linestyle='--', label='1st zero (2.405)')
    ax.axvline(5.5201, color='r', linestyle='--', alpha=0.5, label='2nd zero (5.520)')
    ax.set_xlabel('$2\\pi rf/c$')
    ax.set_ylabel('$J_0(2\\pi rf/c)$')
    ax.set_title('Bessel Function of the First Kind (Order 0)')
    ax.grid(True, alpha=0.3)
    ax.legend()
    ax.set_ylim(-0.5, 1.1)
    
    # 3. 周波数による相関係数の変化
    ax = axes[1, 0]
    frequencies = np.linspace(0.1, 10, 100)
    r = 50  # 観測点間距離
    
    for c in [100, 200, 300, 400]:
        spac_coeff = [j0(2 * np.pi * f * r / c) for f in frequencies]
        ax.plot(frequencies, spac_coeff, label=f'c = {c} m/s')
    
    ax.set_xlabel('Frequency (Hz)')
    ax.set_ylabel('SPAC Coefficient')
    ax.set_title('SPAC Coefficient vs Frequency for Different Velocities')
    ax.grid(True, alpha=0.3)
    ax.legend()
    ax.set_ylim(-0.5, 1.1)
    
    # 4. 簡易推定法のデモ
    ax = axes[1, 1]
    r_values = np.array([10, 20, 50, 100])  # 異なるアレイ半径
    c_true = 250  # 真の位相速度
    
    for r in r_values:
        # 第1ゼロ点の周波数
        f0 = c_true * 2.4048 / (2 * np.pi * r)
        # 簡易推定による位相速度
        c_est = 2.6 * r * f0
        
        ax.scatter(r, f0, s=100, label=f'r={r}m')
        ax.text(r+2, f0, f'$c\\approx${c_est:.0f}m/s', fontsize=9)
    
    ax.set_xlabel('Array Radius (m)')
    ax.set_ylabel('First Zero Frequency (Hz)')
    ax.set_title(f'Simple Estimation Method (True velocity: {c_true} m/s)')
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    plt.tight_layout()
    plt.show()

# 原理の可視化
visualize_spac_principle()

## 3. SPAC法（空間自己相関法）の実装

SPAC法は、アレイ観測データから表面波の位相速度を推定する手法です。

### 3.1 なぜ空間相関から位相速度が推定できるのか？

#### 直感的な理解
2つの観測点で同じ波を観測する場合：
- **観測点間距離が波長の整数倍**：波形が完全に一致（相関係数 = 1）
- **観測点間距離が波長の半整数倍**：波形が逆位相（相関係数 = -1）
- **その他の距離**：中間的な相関値

#### 数学的な表現
等方的な波動場を仮定すると、空間自己相関係数は第1種0次ベッセル関数で表現されます：

$$\rho(r, f) = J_0\left(\frac{2\pi rf}{c(f)}\right)$$

ここで：
- $r$：観測点間距離
- $f$：周波数
- $c(f)$：位相速度
- $J_0$：第1種0次ベッセル関数

### 3.2 簡易推定法
ベッセル関数の第1ゼロ点（$J_0(2.405) \approx 0$）を利用すると：

$$c \approx 2.6 \cdot rf_0$$

ここで $f_0$ は相関係数が初めてゼロになる周波数です。

## 3.5 データ前処理と品質管理

実際の解析では、データの前処理と品質管理が非常に重要です。

### 前処理の手順：
1. **機器応答の補正**
2. **トレンド除去**（線形・多項式）
3. **バンドパスフィルタの適用**
4. **テーパー処理**（窓関数の適用）
5. **過渡的ノイズの除去**

### 品質評価指標：
- **信号対雑音比（SNR）**
- **コヒーレンス**
- **空間自己相関係数の標準偏差**

In [ ]:
class MicrotremorPreprocessor:
    """微動データの前処理クラス"""
    
    def __init__(self, data, fs):
        """
        Parameters:
        - data: 観測データ (n_stations × n_samples)
        - fs: サンプリング周波数
        """
        self.data = np.array(data)
        self.fs = fs
        self.n_stations = data.shape[0]
        self.n_samples = data.shape[1]
        
    def detrend(self, type='linear'):
        """トレンド除去"""
        detrended = np.zeros_like(self.data)
        for i in range(self.n_stations):
            detrended[i, :] = signal.detrend(self.data[i, :], type=type)
        return detrended
    
    def apply_taper(self, alpha=0.05):
        """テーパー処理（Tukey窓）"""
        tapered = np.zeros_like(self.data)
        window = signal.tukey(self.n_samples, alpha=alpha)
        for i in range(self.n_stations):
            tapered[i, :] = self.data[i, :] * window
        return tapered
    
    def bandpass_filter(self, freqmin, freqmax, corners=4):
        """バンドパスフィルタ"""
        nyquist = 0.5 * self.fs
        low = freqmin / nyquist
        high = freqmax / nyquist
        
        b, a = signal.butter(corners, [low, high], btype='band')
        filtered = np.zeros_like(self.data)
        
        for i in range(self.n_stations):
            filtered[i, :] = signal.filtfilt(b, a, self.data[i, :])
            
        return filtered
    
    def remove_transients(self, threshold=3.0):
        """過渡的ノイズの検出と除去"""
        # 各観測点のRMS振幅を計算
        rms = np.sqrt(np.mean(self.data**2, axis=1))
        median_rms = np.median(rms)
        
        # 閾値を超える観測点を検出
        bad_stations = rms > median_rms * threshold
        
        # マスク処理
        cleaned_data = self.data.copy()
        cleaned_data[bad_stations, :] = np.nan
        
        return cleaned_data, bad_stations
    
    def compute_snr(self, signal_band=(1, 5), noise_band=(10, 20)):
        """信号対雑音比（SNR）の計算"""
        # 信号帯域のパワー
        signal_data = self.bandpass_filter(signal_band[0], signal_band[1])
        signal_power = np.mean(signal_data**2, axis=1)
        
        # ノイズ帯域のパワー
        noise_data = self.bandpass_filter(noise_band[0], noise_band[1])
        noise_power = np.mean(noise_data**2, axis=1)
        
        # SNR (dB)
        snr_db = 10 * np.log10(signal_power / noise_power)
        
        return snr_db

# 前処理のデモンストレーション
def demonstrate_preprocessing():
    """前処理の効果を可視化"""
    
    # ノイズを含む合成データの生成
    t = np.linspace(0, 60, 6000)
    fs = 100
    n_stations = 3
    
    # 基本信号
    data = np.zeros((n_stations, len(t)))
    for i in range(n_stations):
        # 信号成分
        data[i, :] = np.sin(2*np.pi*2*t) + 0.5*np.sin(2*np.pi*5*t)
        # トレンド追加
        data[i, :] += 0.001 * t
        # スパイクノイズ追加（観測点2のみ）
        if i == 1:
            data[i, 3000:3010] += 10
        # ランダムノイズ
        data[i, :] += 0.2 * np.random.randn(len(t))
    
    # 前処理
    preprocessor = MicrotremorPreprocessor(data, fs)
    
    # 各ステップの適用
    data_detrended = preprocessor.detrend()
    data_filtered = preprocessor.bandpass_filter(0.5, 10)
    data_tapered = preprocessor.apply_taper(alpha=0.1)
    
    # 可視化
    fig, axes = plt.subplots(4, 1, figsize=(12, 10))
    
    # 元データ
    ax = axes[0]
    for i in range(n_stations):
        ax.plot(t[:1000], data[i, :1000] + i*5, label=f'Station {i}')
    ax.set_ylabel('Original Data')
    ax.set_title('Preprocessing Steps')
    ax.legend()
    
    # トレンド除去後
    ax = axes[1]
    preprocessor.data = data_detrended
    for i in range(n_stations):
        ax.plot(t[:1000], data_detrended[i, :1000] + i*5)
    ax.set_ylabel('After Detrending')
    
    # フィルタ適用後
    ax = axes[2]
    for i in range(n_stations):
        ax.plot(t[:1000], data_filtered[i, :1000] + i*5)
    ax.set_ylabel('After Bandpass')
    
    # SNR計算
    ax = axes[3]
    preprocessor.data = data_filtered
    snr = preprocessor.compute_snr()
    ax.bar(range(n_stations), snr)
    ax.set_xlabel('Station Number')
    ax.set_ylabel('SNR (dB)')
    ax.set_title('Signal-to-Noise Ratio')
    ax.axhline(10, color='r', linestyle='--', label='Recommended Threshold')
    ax.legend()
    
    for ax in axes[:-1]:
        ax.set_xlim(0, 10)
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# デモンストレーションの実行
demonstrate_preprocessing()

In [ ]:
class SPACAnalysis:
    """SPAC法による解析クラス"""
    
    def __init__(self, data, fs, array_radius):
        """
        Parameters:
        - data: 観測データ (n_stations × n_samples)
        - fs: サンプリング周波数
        - array_radius: アレイ半径
        """
        self.data = data
        self.fs = fs
        self.radius = array_radius
        self.n_stations = data.shape[0]
        
    def compute_spac_coefficient(self, window_length=20, overlap=0.5):
        """空間自己相関係数を計算"""
        
        # 窓の設定
        nperseg = int(window_length * self.fs)
        noverlap = int(nperseg * overlap)
        
        # 中心点と周辺点のデータ
        center_data = self.data[0, :]  # 中心点
        
        # 各周波数での空間自己相関係数を格納
        spac_coeffs = []
        
        # 各周辺観測点との相関を計算
        for i in range(1, self.n_stations):
            # クロススペクトル密度の計算
            f, Pxy = signal.csd(center_data, self.data[i, :], 
                               fs=self.fs, nperseg=nperseg, noverlap=noverlap)
            
            # オートスペクトル密度の計算
            _, Pxx = signal.welch(center_data, fs=self.fs, 
                                 nperseg=nperseg, noverlap=noverlap)
            _, Pyy = signal.welch(self.data[i, :], fs=self.fs, 
                                 nperseg=nperseg, noverlap=noverlap)
            
            # 空間自己相関係数
            spac = np.real(Pxy) / np.sqrt(Pxx * Pyy)
            spac_coeffs.append(spac)
        
        # 平均化
        self.frequencies = f
        self.spac_mean = np.mean(spac_coeffs, axis=0)
        self.spac_std = np.std(spac_coeffs, axis=0)
        
        return self.frequencies, self.spac_mean, self.spac_std
    
    def estimate_phase_velocity(self, freq_range=(0.5, 10)):
        """ベッセル関数フィッティングによる位相速度推定"""
        
        # 解析する周波数範囲を限定
        freq_mask = (self.frequencies >= freq_range[0]) & (self.frequencies <= freq_range[1])
        freq_analysis = self.frequencies[freq_mask]
        spac_analysis = self.spac_mean[freq_mask]
        
        phase_velocities = []
        
        for f, spac_val in zip(freq_analysis, spac_analysis):
            if f == 0 or abs(spac_val) > 1:
                phase_velocities.append(np.nan)
                continue
                
            # ベッセル関数の零点を探索
            # J0(2πfr/c) = spac_val を満たすcを探す
            def bessel_func(c):
                return j0(2 * np.pi * f * self.radius / c) - spac_val
            
            # 初期値の設定（探索範囲）
            c_min = 50  # 最小位相速度 (m/s)
            c_max = 1000  # 最大位相速度 (m/s)
            
            try:
                from scipy.optimize import brentq
                c = brentq(bessel_func, c_min, c_max)
                phase_velocities.append(c)
            except:
                phase_velocities.append(np.nan)
        
        self.phase_velocities = np.array(phase_velocities)
        self.freq_analysis = freq_analysis
        
        return freq_analysis, self.phase_velocities

## 4. 実践：合成データでのSPAC解析

実際の地下構造を想定した合成データを生成し、SPAC法を適用してみましょう。

In [ ]:
# 改良されたSPAC解析クラス（品質管理機能付き）
class SPACAnalysisWithQC(SPACAnalysis):
    """品質管理機能を追加したSPAC解析クラス"""
    
    def compute_spac_with_coherence(self, window_length=20, overlap=0.5):
        """空間自己相関係数とコヒーレンスを同時計算"""
        
        # 窓の設定
        nperseg = int(window_length * self.fs)
        noverlap = int(nperseg * overlap)
        
        # 中心点と周辺点のデータ
        center_data = self.data[0, :]
        
        # 各周波数での結果を格納
        spac_coeffs = []
        coherences = []
        
        # 各周辺観測点との相関を計算
        for i in range(1, self.n_stations):
            # クロススペクトル密度の計算
            f, Pxy = signal.csd(center_data, self.data[i, :], 
                               fs=self.fs, nperseg=nperseg, noverlap=noverlap)
            
            # オートスペクトル密度の計算
            _, Pxx = signal.welch(center_data, fs=self.fs, 
                                 nperseg=nperseg, noverlap=noverlap)
            _, Pyy = signal.welch(self.data[i, :], fs=self.fs, 
                                 nperseg=nperseg, noverlap=noverlap)
            
            # 空間自己相関係数
            spac = np.real(Pxy) / np.sqrt(Pxx * Pyy)
            spac_coeffs.append(spac)
            
            # コヒーレンス
            coherence = np.abs(Pxy)**2 / (Pxx * Pyy)
            coherences.append(coherence)
        
        # 結果の保存
        self.frequencies = f
        self.spac_mean = np.mean(spac_coeffs, axis=0)
        self.spac_std = np.std(spac_coeffs, axis=0)
        self.coherence_mean = np.mean(coherences, axis=0)
        self.coherence_std = np.std(coherences, axis=0)
        
        return f, self.spac_mean, self.spac_std, self.coherence_mean
    
    def quality_check(self, min_coherence=0.8):
        """データ品質チェック"""
        quality_mask = np.ones_like(self.frequencies, dtype=bool)
        
        # コヒーレンスチェック
        quality_mask &= (self.coherence_mean >= min_coherence)
        
        # SPAC係数の範囲チェック
        quality_mask &= (np.abs(self.spac_mean) <= 1.0)
        
        # 標準偏差が大きすぎる点を除外
        quality_mask &= (self.spac_std < 0.3)
        
        # 品質スコア
        quality_score = np.sum(quality_mask) / len(quality_mask)
        
        return quality_mask, quality_score
    
    def plot_results_with_qc(self):
        """品質管理結果を含むプロット"""
        fig, axes = plt.subplots(3, 1, figsize=(10, 12))
        
        # SPAC係数
        ax = axes[0]
        ax.plot(self.frequencies, self.spac_mean, 'b-', linewidth=2, label='SPAC Coefficient')
        ax.fill_between(self.frequencies, 
                       self.spac_mean - self.spac_std, 
                       self.spac_mean + self.spac_std, 
                       alpha=0.3, color='blue', label='±1 std')
        ax.set_xlabel('Frequency (Hz)')
        ax.set_ylabel('SPAC Coefficient')
        ax.set_title('Spatial Autocorrelation Coefficient')
        ax.grid(True, alpha=0.3)
        ax.legend()
        ax.set_ylim(-1.2, 1.2)
        
        # コヒーレンス
        ax = axes[1]
        ax.plot(self.frequencies, self.coherence_mean, 'g-', linewidth=2, label='Coherence')
        ax.fill_between(self.frequencies, 
                       self.coherence_mean - self.coherence_std, 
                       self.coherence_mean + self.coherence_std, 
                       alpha=0.3, color='green')
        ax.axhline(0.8, color='r', linestyle='--', label='Quality Threshold')
        ax.set_xlabel('Frequency (Hz)')
        ax.set_ylabel('Coherence')
        ax.set_title('Coherence between Stations')
        ax.grid(True, alpha=0.3)
        ax.legend()
        ax.set_ylim(0, 1.2)
        
        # 品質マスク
        ax = axes[2]
        quality_mask, quality_score = self.quality_check()
        ax.scatter(self.frequencies[quality_mask], self.spac_mean[quality_mask], 
                  c='g', s=20, label='Good Quality')
        ax.scatter(self.frequencies[~quality_mask], self.spac_mean[~quality_mask], 
                  c='r', s=20, alpha=0.5, label='Poor Quality')
        ax.set_xlabel('Frequency (Hz)')
        ax.set_ylabel('SPAC Coefficient')
        ax.set_title(f'Quality Control Result (Score: {quality_score:.2%})')
        ax.grid(True, alpha=0.3)
        ax.legend()
        ax.set_ylim(-1.2, 1.2)
        
        plt.tight_layout()
        plt.show()

# 品質管理付きSPAC解析のデモ
spac_qc = SPACAnalysisWithQC(array_data, fs=100, array_radius=50)
freq, spac_coeff, spac_std, coherence = spac_qc.compute_spac_with_coherence()
spac_qc.plot_results_with_qc()

In [ ]:
# 合成アレイデータの生成
def generate_array_data(n_stations=5, array_radius=50, duration=300, fs=100):
    """
    表面波を含む合成アレイデータを生成
    """
    t = np.arange(0, duration, 1/fs)
    data = np.zeros((n_stations, len(t)))
    
    # 表面波の特性（周波数と位相速度）
    frequencies = [1.0, 2.0, 3.0, 5.0]
    phase_velocities = [300, 250, 200, 150]  # 深さとともに速度増加を模擬
    amplitudes = [1.0, 0.7, 0.5, 0.3]
    
    # 各観測点の位置（円形アレイ）
    angles = np.linspace(0, 2*np.pi, n_stations-1, endpoint=False)
    x_stations = [0] + [array_radius * np.cos(a) for a in angles]
    y_stations = [0] + [array_radius * np.sin(a) for a in angles]
    
    # 各周波数成分について
    for f, c, amp in zip(frequencies, phase_velocities, amplitudes):
        # 波の進行方向（ランダム）
        theta = np.random.rand() * 2 * np.pi
        kx = 2 * np.pi * f / c * np.cos(theta)
        ky = 2 * np.pi * f / c * np.sin(theta)
        
        # 各観測点での波形
        for i, (x, y) in enumerate(zip(x_stations, y_stations)):
            phase = kx * x + ky * y
            data[i, :] += amp * np.sin(2 * np.pi * f * t - phase)
    
    # ノイズを追加
    data += 0.1 * np.random.randn(*data.shape)
    
    return data, t

# データ生成
array_data, time = generate_array_data(n_stations=5, array_radius=50)

# 生成したデータの可視化
plt.figure(figsize=(12, 8))
for i in range(5):
    plt.subplot(5, 1, i+1)
    plt.plot(time[:500], array_data[i, :500])
    plt.ylabel(f'Station {i}')
    if i == 0:
        plt.title('Synthetic Array Data (First 5 seconds)')
    if i == 4:
        plt.xlabel('Time (s)')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# SPAC解析の実行
spac = SPACAnalysis(array_data, fs=100, array_radius=50)

# 空間自己相関係数の計算
freq, spac_coeff, spac_std = spac.compute_spac_coefficient(window_length=20)

# 結果の可視化
plt.figure(figsize=(12, 5))

# 空間自己相関係数
plt.subplot(1, 2, 1)
plt.plot(freq, spac_coeff, 'b-', linewidth=2, label='SPAC Coefficient')
plt.fill_between(freq, spac_coeff - spac_std, spac_coeff + spac_std, 
                 alpha=0.3, color='blue', label='±1 std')
plt.xlabel('Frequency (Hz)')
plt.ylabel('SPAC Coefficient')
plt.title('Spatial Autocorrelation Coefficient')
plt.grid(True, alpha=0.3)
plt.xlim(0, 10)
plt.ylim(-0.5, 1.0)
plt.legend()

# ベッセル関数との比較
plt.subplot(1, 2, 2)
# いくつかの位相速度でベッセル関数を描画
velocities = [150, 200, 250, 300]
for v in velocities:
    bessel_values = [j0(2 * np.pi * f * 50 / v) for f in freq]
    plt.plot(freq, bessel_values, '--', alpha=0.5, label=f'v={v} m/s')

plt.plot(freq, spac_coeff, 'b-', linewidth=2, label='Observed')
plt.xlabel('Frequency (Hz)')
plt.ylabel('SPAC Coefficient')
plt.title('Comparison with Bessel Functions')
plt.grid(True, alpha=0.3)
plt.xlim(0, 10)
plt.ylim(-0.5, 1.0)
plt.legend()

plt.tight_layout()
plt.show()

### 理論分散曲線の計算

簡易的な2層モデルでは、位相速度は以下のような特性を示します：

- 高周波数（短波長）：第1層のS波速度$V_{s1}$に近づく
- 低周波数（長波長）：第2層のS波速度$V_{s2}$に近づく

波長$\lambda$と層厚$h_1$の関係により、次のような遷移が起こります：

$$c(f) = V_{s1} + (V_{s2} - V_{s1}) \cdot \left(1 - \exp\left(-\frac{2h_1}{\lambda}\right)\right)$$

In [ ]:
# 位相速度の推定
freq_analysis, phase_vel = spac.estimate_phase_velocity(freq_range=(0.5, 6))

# 分散曲線の表示
plt.figure(figsize=(10, 6))
plt.scatter(freq_analysis, phase_vel, s=50, c='blue', alpha=0.7, label='Estimated')

# 真の値をプロット（検証用）
true_freqs = [1.0, 2.0, 3.0, 5.0]
true_vels = [300, 250, 200, 150]
plt.plot(true_freqs, true_vels, 'r--', linewidth=2, marker='o', 
         markersize=8, label='True values')

plt.xlabel('Frequency (Hz)')
plt.ylabel('Phase Velocity (m/s)')
plt.title('Dispersion Curve from SPAC Analysis')
plt.grid(True, alpha=0.3)
plt.legend()
plt.xlim(0, 7)
plt.ylim(0, 400)

# 対数軸での表示も追加
plt.figure(figsize=(10, 6))
plt.loglog(1/freq_analysis, phase_vel, 'bo-', markersize=6, label='Estimated')
plt.loglog(1/np.array(true_freqs), true_vels, 'r--', linewidth=2, 
           marker='s', markersize=8, label='True values')
plt.xlabel('Period (s)')
plt.ylabel('Phase Velocity (m/s)')
plt.title('Dispersion Curve (Log-Log Scale)')
plt.grid(True, which="both", alpha=0.3)
plt.legend()
plt.xlim(0.1, 10)
plt.ylim(100, 500)

plt.show()

## 5. 地下構造の推定

分散曲線から地下構造（S波速度構造）を推定します。
簡単な2層モデルを例に、理論分散曲線の計算方法を示します。

In [ ]:
def simple_dispersion_curve(frequencies, h1, vs1, vs2):
    """
    簡単な2層モデルの理論分散曲線を計算
    
    Parameters:
    - frequencies: 周波数配列
    - h1: 第1層の層厚 (m)
    - vs1: 第1層のS波速度 (m/s)
    - vs2: 第2層（半無限）のS波速度 (m/s)
    
    Returns:
    - phase_velocities: 位相速度配列
    """
    phase_velocities = []
    
    for f in frequencies:
        # 波長
        lambda_min = vs1 / f
        lambda_max = vs2 / f
        
        # 簡易的な推定式（実際はより複雑）
        # 浅い深度では vs1 に近く、深い深度では vs2 に近づく
        depth_factor = 1 - np.exp(-2 * h1 / lambda_min)
        c = vs1 + (vs2 - vs1) * depth_factor
        
        phase_velocities.append(c)
    
    return np.array(phase_velocities)

# パラメータスタディ
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 周波数配列
f_theory = np.linspace(0.1, 10, 100)

# 1. 層厚の影響
ax = axes[0, 0]
for h in [5, 10, 20, 40]:
    c_theory = simple_dispersion_curve(f_theory, h, 150, 300)
    ax.plot(f_theory, c_theory, label=f'h1={h}m')
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Phase Velocity (m/s)')
ax.set_title('Effect of Layer Thickness')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. 第1層速度の影響
ax = axes[0, 1]
for vs1 in [100, 150, 200, 250]:
    c_theory = simple_dispersion_curve(f_theory, 20, vs1, 400)
    ax.plot(f_theory, c_theory, label=f'Vs1={vs1}m/s')
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Phase Velocity (m/s)')
ax.set_title('Effect of Layer 1 Velocity')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. 第2層速度の影響
ax = axes[1, 0]
for vs2 in [300, 400, 500, 600]:
    c_theory = simple_dispersion_curve(f_theory, 20, 150, vs2)
    ax.plot(f_theory, c_theory, label=f'Vs2={vs2}m/s')
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Phase Velocity (m/s)')
ax.set_title('Effect of Layer 2 Velocity')
ax.legend()
ax.grid(True, alpha=0.3)

# 4. 観測データとのフィッティング例
ax = axes[1, 1]
# 観測データ（先ほどの結果を使用）
valid_mask = ~np.isnan(phase_vel)
ax.scatter(freq_analysis[valid_mask], phase_vel[valid_mask], 
          s=50, c='blue', alpha=0.7, label='Observed')

# いくつかのモデルを試す
models = [
    (10, 180, 280, 'Model 1'),
    (15, 160, 300, 'Model 2'),
    (20, 150, 320, 'Model 3')
]

for h, vs1, vs2, label in models:
    c_theory = simple_dispersion_curve(freq_analysis, h, vs1, vs2)
    ax.plot(freq_analysis, c_theory, '--', linewidth=2, label=label)

ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Phase Velocity (m/s)')
ax.set_title('Model Fitting Example')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 6)
ax.set_ylim(100, 400)

plt.tight_layout()
plt.show()

## 6. 実践演習

以下の演習問題に取り組んでみましょう：

### 演習1：ノイズの影響
異なるノイズレベルでのSPAC解析の安定性を調べてください。

### 演習2：アレイサイズの検討
異なるアレイサイズ（半径）での解析可能周波数範囲を調べてください。

### 演習3：観測点数の影響
観測点数を変えた場合の解析精度を評価してください。

In [ ]:
# 演習1の解答例：ノイズの影響
def analyze_noise_effect(noise_levels=[0.01, 0.05, 0.1, 0.2, 0.5]):
    """異なるノイズレベルでのSPAC解析"""
    
    fig, axes = plt.subplots(2, len(noise_levels), figsize=(15, 8))
    
    for idx, noise_level in enumerate(noise_levels):
        # データ生成（ノイズレベルを変更）
        t = np.arange(0, 300, 0.01)
        data = np.zeros((5, len(t)))
        
        # 表面波成分
        frequencies = [1.0, 2.0, 3.0, 5.0]
        phase_velocities = [300, 250, 200, 150]
        
        for i in range(5):
            for f, c in zip(frequencies, phase_velocities):
                phase = 2 * np.pi * f * i * 50 / (4 * c)  # 簡易的な位相差
                data[i, :] += np.sin(2 * np.pi * f * t - phase)
        
        # ノイズ追加
        data += noise_level * np.random.randn(*data.shape)
        
        # SPAC解析
        spac = SPACAnalysis(data, fs=100, array_radius=50)
        freq, spac_coeff, _ = spac.compute_spac_coefficient()
        
        # プロット
        ax1 = axes[0, idx]
        ax1.plot(t[:100], data[0, :100])
        ax1.set_title(f'Noise Level: {noise_level}')
        ax1.set_xlabel('Time (s)')
        ax1.set_ylabel('Amplitude')
        
        ax2 = axes[1, idx]
        ax2.plot(freq, spac_coeff)
        ax2.set_xlabel('Frequency (Hz)')
        ax2.set_ylabel('SPAC Coefficient')
        ax2.set_xlim(0, 10)
        ax2.set_ylim(-0.5, 1)
        ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# 演習1の実行
print("Exercise 1: Effect of Noise Level")
analyze_noise_effect()

## 8. 探査深度の理論的背景

### 8.1 なぜ探査深度が λ/2 ～ λ/3 なのか？

表面波（レイリー波）の振幅は深さとともに指数関数的に減衰します。探査深度が波長の約1/2～1/3になる理由を理解しましょう。

### 8.2 レイリー波の変位分布

レイリー波の変位は深さ方向に以下のように表現されます：

水平成分：
$$u_x(z) = A \left[ e^{-k\alpha z} - \frac{2\alpha\beta}{1+\beta^2} e^{-k\beta z} \right]$$

鉛直成分：
$$u_z(z) = A \frac{i\alpha}{k} \left[ e^{-k\alpha z} - \frac{2}{1+\beta^2} e^{-k\beta z} \right]$$

ここで：
- $k = 2\pi/\lambda$：波数
- $\alpha = \sqrt{1 - (c/v_p)^2}$
- $\beta = \sqrt{1 - (c/v_s)^2}$

### 8.3 感度カーネル

位相速度は各深さのS波速度の重み付き平均として表現でき、この重み関数（感度カーネル）は深さ $z = \lambda/3$ 付近で最大となります。

In [ ]:
def visualize_exploration_depth():
    """レイリー波の変位分布と探査深度の関係を可視化"""
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # パラメータ設定
    wavelengths = [10, 20, 50, 100]  # 波長 (m)
    vs = 200  # S波速度 (m/s)
    vp = vs * 1.73  # P波速度（ポアソン比0.25）
    c = 0.92 * vs  # レイリー波速度（近似）
    
    # 1. 変位の深さ分布
    ax = axes[0, 0]
    for lambda_val in wavelengths:
        z = np.linspace(0, 2*lambda_val, 1000)
        k = 2 * np.pi / lambda_val
        alpha = np.sqrt(1 - (c/vp)**2)
        beta = np.sqrt(1 - (c/vs)**2)
        
        # 水平変位（正規化）
        ux = np.exp(-k*alpha*z) - (2*alpha*beta)/(1+beta**2) * np.exp(-k*beta*z)
        ux = ux / np.max(np.abs(ux))
        
        ax.plot(ux, z/lambda_val, label=f'$\\lambda$={lambda_val}m')
        
    ax.invert_yaxis()
    ax.set_xlabel('Normalized Displacement')
    ax.set_ylabel('Depth/$\\lambda$')
    ax.set_title('Rayleigh Wave Horizontal Displacement Distribution')
    ax.grid(True, alpha=0.3)
    ax.legend()
    ax.axhline(1/3, color='r', linestyle='--', alpha=0.5, label='$\\lambda$/3')
    ax.axhline(1/2, color='r', linestyle='--', alpha=0.5, label='$\\lambda$/2')
    
    # 2. エネルギー分布
    ax = axes[0, 1]
    for lambda_val in wavelengths:
        z = np.linspace(0, 2*lambda_val, 1000)
        k = 2 * np.pi / lambda_val
        alpha = np.sqrt(1 - (c/vp)**2)
        beta = np.sqrt(1 - (c/vs)**2)
        
        # エネルギー（変位の2乗に比例）
        ux = np.exp(-k*alpha*z) - (2*alpha*beta)/(1+beta**2) * np.exp(-k*beta*z)
        energy = ux**2
        
        # 累積エネルギー
        cumulative_energy = np.cumsum(energy) / np.sum(energy)
        
        ax.plot(cumulative_energy, z/lambda_val, label=f'$\\lambda$={lambda_val}m')
        
    ax.invert_yaxis()
    ax.set_xlabel('Cumulative Energy Ratio')
    ax.set_ylabel('Depth/$\\lambda$')
    ax.set_title('Energy Distribution with Depth')
    ax.grid(True, alpha=0.3)
    ax.legend()
    ax.axhline(1/3, color='r', linestyle='--', alpha=0.5)
    ax.axhline(1/2, color='r', linestyle='--', alpha=0.5)
    ax.axvline(0.7, color='g', linestyle='--', alpha=0.5, label='70%')
    
    # 3. 感度カーネル
    ax = axes[1, 0]
    z_norm = np.linspace(0, 2, 1000)
    
    # 簡易的な感度カーネル（実際はより複雑）
    sensitivity = z_norm * np.exp(-3*z_norm)
    sensitivity = sensitivity / np.max(sensitivity)
    
    ax.plot(sensitivity, z_norm)
    ax.invert_yaxis()
    ax.set_xlabel('Sensitivity')
    ax.set_ylabel('Depth/$\\lambda$')
    ax.set_title('Sensitivity Kernel for S-wave Velocity (Conceptual)')
    ax.grid(True, alpha=0.3)
    ax.axhline(1/3, color='r', linestyle='--', alpha=0.5, label='$\\lambda$/3 (Max Sensitivity)')
    ax.fill_betweenx(z_norm, 0, sensitivity, alpha=0.3)
    
    # 4. 周波数と探査深度の関係
    ax = axes[1, 1]
    frequencies = np.logspace(-1, 1.5, 50)  # 0.1-30 Hz
    vs_values = [100, 200, 300, 500]
    
    for vs in vs_values:
        wavelengths = vs / frequencies
        depth_min = wavelengths / 3
        depth_max = wavelengths / 2
        
        ax.fill_between(frequencies, depth_min, depth_max, alpha=0.3, label=f'$V_s$={vs}m/s')
        ax.loglog(frequencies, depth_min, '--', alpha=0.5)
        ax.loglog(frequencies, depth_max, '--', alpha=0.5)
    
    ax.set_xlabel('Frequency (Hz)')
    ax.set_ylabel('Exploration Depth (m)')
    ax.set_title('Frequency vs Exploration Depth')
    ax.grid(True, which="both", alpha=0.3)
    ax.legend()
    ax.set_xlim(0.1, 30)
    ax.set_ylim(1, 1000)
    
    plt.tight_layout()
    plt.show()

# 可視化の実行
visualize_exploration_depth()

## 7. まとめと実践のポイント

### 重要なポイント
1. **アレイサイズの選択**：対象とする周波数帯域に応じて適切に選ぶ
2. **観測時間**：統計的に安定した結果を得るため、20-30分以上の観測を推奨
3. **データ品質管理**：過渡的なノイズの除去、機器の同期が重要
4. **解の非唯一性**：異なる地下構造モデルが同じ分散曲線を示す可能性がある

### 次のステップ
1. 実データの解析に挑戦
2. FK法など他の解析手法の習得
3. 逆解析手法（遺伝的アルゴリズムなど）の実装
4. 3次元構造への拡張

### 参考資料
- Okada, H. (2003): The Microtremor Survey Method
- 物理探査学会 (2008): 物理探査ハンドブック
- Geopsy プロジェクト: http://www.geopsy.org/

## 5. FK法 (Frequency-Wavenumber Method)

FK法は、アレイ観測データを周波数-波数領域で解析し、表面波の伝播方向と位相速度を同時に推定する手法です。

In [ ]:
def visualize_fk_method():
    """FK法の原理を可視化"""
    fig = plt.figure(figsize=(16, 12))
    
    # 1. 波動場の例
    ax1 = plt.subplot(2, 3, 1)
    x = np.linspace(-100, 100, 200)
    y = np.linspace(-100, 100, 200)
    X, Y = np.meshgrid(x, y)
    
    # 平面波 (wave coming from NE direction)
    k_x, k_y = 0.05, 0.05  # wavenumber components
    wave_field = np.sin(k_x * X + k_y * Y)
    
    im1 = ax1.contourf(X, Y, wave_field, levels=20, cmap='RdBu')
    ax1.scatter([0, 50, 0, -50, 0], [0, 0, 50, 0, -50], 
                s=100, c='black', marker='^', label='Array stations')
    ax1.arrow(0, 0, 30, 30, head_width=5, head_length=5, fc='green', ec='green')
    ax1.text(35, 35, 'Wave direction', fontsize=10)
    ax1.set_xlabel('X (m)')
    ax1.set_ylabel('Y (m)')
    ax1.set_title('Wave Field at Array')
    ax1.axis('equal')
    ax1.grid(True, alpha=0.3)
    plt.colorbar(im1, ax=ax1, label='Amplitude')
    
    # 2. FK spectrum
    ax2 = plt.subplot(2, 3, 2)
    kx = np.linspace(-0.1, 0.1, 100)
    ky = np.linspace(-0.1, 0.1, 100)
    KX, KY = np.meshgrid(kx, ky)
    
    # Simulated FK power spectrum
    fk_power = np.exp(-50 * ((KX - k_x)**2 + (KY - k_y)**2))
    
    im2 = ax2.contourf(KX, KY, fk_power, levels=20, cmap='hot')
    ax2.scatter([k_x], [k_y], s=100, c='blue', marker='*', 
                label=f'Peak at k=({k_x:.2f}, {k_y:.2f})')
    ax2.set_xlabel(r'$k_x$ (rad/m)')
    ax2.set_ylabel(r'$k_y$ (rad/m)')
    ax2.set_title('FK Power Spectrum')
    ax2.axis('equal')
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    plt.colorbar(im2, ax=ax2, label='Power')
    
    # 3. Slowness domain
    ax3 = plt.subplot(2, 3, 3)
    sx = kx / (2 * np.pi * 5)  # Convert to slowness for f=5 Hz
    sy = ky / (2 * np.pi * 5)
    SX, SY = np.meshgrid(sx, sy)
    
    im3 = ax3.contourf(SX * 1000, SY * 1000, fk_power, levels=20, cmap='hot')
    
    # Add velocity circles
    for v in [100, 200, 300, 400]:
        circle = plt.Circle((0, 0), 1000/v, fill=False, color='white', 
                          linestyle='--', alpha=0.5)
        ax3.add_patch(circle)
        ax3.text(0, 1000/v + 0.5, f'{v} m/s', ha='center', 
                color='white', fontsize=8)
    
    ax3.set_xlabel('Slowness X (s/km)')
    ax3.set_ylabel('Slowness Y (s/km)')
    ax3.set_title('Slowness Domain (f = 5 Hz)')
    ax3.set_xlim(-10, 10)
    ax3.set_ylim(-10, 10)
    ax3.axis('equal')
    ax3.grid(True, alpha=0.3)
    plt.colorbar(im3, ax=ax3, label='Power')
    
    # 4. Array response function
    ax4 = plt.subplot(2, 3, 4)
    # Array coordinates (5-station array)
    stations = np.array([[0, 0], [50, 0], [0, 50], [-50, 0], [0, -50]])
    
    # Compute array response
    response = np.zeros_like(fk_power)
    for i in range(len(kx)):
        for j in range(len(ky)):
            steering = np.exp(-1j * (kx[i] * stations[:, 0] + 
                                    ky[j] * stations[:, 1]))
            response[j, i] = np.abs(np.sum(steering))**2 / len(stations)**2
    
    im4 = ax4.contourf(KX, KY, response, levels=20, cmap='viridis')
    ax4.set_xlabel(r'$k_x$ (rad/m)')
    ax4.set_ylabel(r'$k_y$ (rad/m)')
    ax4.set_title('Array Response Function')
    ax4.axis('equal')
    ax4.grid(True, alpha=0.3)
    plt.colorbar(im4, ax=ax4, label='Response')
    
    # 5. Phase velocity estimation
    ax5 = plt.subplot(2, 3, 5)
    frequencies = np.logspace(0, 1.5, 30)
    
    # Simulated dispersion curves
    true_velocity = 200 * frequencies**0.1
    fk_velocity = true_velocity * (1 + 0.05 * np.random.randn(len(frequencies)))
    
    ax5.scatter(frequencies, fk_velocity, s=50, alpha=0.7, 
                label='FK estimates', color='red')
    ax5.plot(frequencies, true_velocity, 'k--', 
             label='True dispersion', linewidth=2)
    
    ax5.set_xlabel('Frequency (Hz)')
    ax5.set_ylabel('Phase Velocity (m/s)')
    ax5.set_title('Dispersion Curve from FK Analysis')
    ax5.set_xscale('log')
    ax5.grid(True, alpha=0.3, which='both')
    ax5.legend()
    
    # 6. Beamforming concept
    ax6 = plt.subplot(2, 3, 6)
    theta = np.linspace(0, 2*np.pi, 100)
    
    # Beam patterns for different frequencies
    r1 = 1 + 0.3 * np.cos(5 * theta)  # Main lobe
    r2 = 1 + 0.2 * np.cos(3 * theta) + 0.1 * np.cos(7 * theta)  # With side lobes
    
    ax6.plot(theta, r1, label='Good resolution', linewidth=2)
    ax6.plot(theta, r2, label='With side lobes', linewidth=2, alpha=0.7)
    ax6.fill_between(theta, 0, r1, alpha=0.3)
    ax6.fill_between(theta, 0, r2, alpha=0.2)
    
    ax6.set_xlabel('Azimuth (rad)')
    ax6.set_ylabel('Beam Power')
    ax6.set_title('Beamforming Resolution')
    ax6.grid(True, alpha=0.3)
    ax6.legend()
    
    plt.tight_layout()
    plt.show()

# 実行
visualize_fk_method()

## 6. トラブルシューティングと実践的な考慮事項

微動解析でよく遭遇する問題とその対策を可視化します。

In [ ]:
def visualize_troubleshooting():
    """一般的な問題と対策を可視化"""
    fig = plt.figure(figsize=(16, 12))
    
    # 1. 低周波数でのSPAC係数の不安定性
    ax1 = plt.subplot(2, 3, 1)
    r = 50  # array radius
    frequencies = np.logspace(-1, 1.5, 100)
    
    # Different velocity models
    c1 = 200  # constant velocity
    c2 = 150 * frequencies**0.15  # dispersive
    
    # SPAC coefficients
    spac1 = [j0(2 * np.pi * f * r / c1) for f in frequencies]
    spac2 = [j0(2 * np.pi * f * r / c2[i]) for i, f in enumerate(frequencies)]
    
    # Mark problematic region
    problem_freq = c1 / (2 * r)  # λ = 2r
    ax1.axvspan(0, problem_freq, alpha=0.3, color='red', 
                label=f'Unstable region (f < {problem_freq:.1f} Hz)')
    
    ax1.plot(frequencies, spac1, 'b-', linewidth=2, label='Non-dispersive')
    ax1.plot(frequencies, spac2, 'g--', linewidth=2, label='Dispersive')
    ax1.axhline(y=0, color='k', linestyle=':', alpha=0.5)
    
    ax1.set_xlabel('Frequency (Hz)')
    ax1.set_ylabel('SPAC Coefficient')
    ax1.set_title(f'Low-frequency Instability (r = {r} m)')
    ax1.set_xscale('log')
    ax1.set_xlim([0.1, 30])
    ax1.set_ylim([-0.5, 1.1])
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # 2. 高周波数でのコヒーレンス低下
    ax2 = plt.subplot(2, 3, 2)
    frequencies = np.linspace(0.1, 30, 100)
    
    # Coherence model (decreasing with frequency)
    coherence_ideal = np.ones_like(frequencies)
    coherence_real = np.exp(-frequencies / 10) * (0.9 + 0.1 * np.random.rand(len(frequencies)))
    
    # Quality threshold
    threshold = 0.8
    ax2.axhline(y=threshold, color='r', linestyle='--', 
                label=f'Quality threshold ({threshold})')
    
    ax2.fill_between(frequencies, 0, coherence_real, 
                     where=(coherence_real >= threshold), 
                     alpha=0.3, color='green', label='Good quality')
    ax2.fill_between(frequencies, 0, coherence_real, 
                     where=(coherence_real < threshold), 
                     alpha=0.3, color='red', label='Poor quality')
    
    ax2.plot(frequencies, coherence_ideal, 'k:', label='Ideal')
    ax2.plot(frequencies, coherence_real, 'b-', linewidth=2, label='Observed')
    
    ax2.set_xlabel('Frequency (Hz)')
    ax2.set_ylabel('Coherence')
    ax2.set_title('High-frequency Coherence Degradation')
    ax2.set_ylim([0, 1.1])
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    
    # 3. 分散曲線の不連続
    ax3 = plt.subplot(2, 3, 3)
    f1 = np.linspace(0.5, 5, 20)
    f2 = np.linspace(4, 15, 30)
    f3 = np.linspace(12, 30, 25)
    
    # Velocities with discontinuities
    v1 = 180 * f1**0.1
    v2 = 220 * f2**0.08  # Jump
    v3 = 250 * f3**0.05  # Another jump
    
    # Show problematic points
    ax3.scatter(f1, v1, s=50, alpha=0.7, label='Array 1 (r=100m)')
    ax3.scatter(f2, v2, s=50, alpha=0.7, label='Array 2 (r=50m)')
    ax3.scatter(f3, v3, s=50, alpha=0.7, label='Array 3 (r=20m)')
    
    # Highlight discontinuities
    ax3.axvspan(4, 5, alpha=0.2, color='red')
    ax3.axvspan(12, 15, alpha=0.2, color='red')
    ax3.text(4.5, 300, 'Discontinuity', rotation=90, va='center', color='red')
    ax3.text(13.5, 300, 'Discontinuity', rotation=90, va='center', color='red')
    
    ax3.set_xlabel('Frequency (Hz)')
    ax3.set_ylabel('Phase Velocity (m/s)')
    ax3.set_title('Dispersion Curve Discontinuities')
    ax3.grid(True, alpha=0.3)
    ax3.legend()
    
    # 4. ノイズの影響
    ax4 = plt.subplot(2, 3, 4)
    t = np.linspace(0, 10, 1000)
    
    # Clean signal
    clean = np.sin(2 * np.pi * 2 * t) + 0.5 * np.sin(2 * np.pi * 5 * t)
    
    # Add different types of noise
    white_noise = clean + 0.3 * np.random.randn(len(t))
    transient = clean.copy()
    transient[400:450] += 5 * np.sin(2 * np.pi * 20 * (t[400:450] - t[400]))
    
    ax4.plot(t, clean, 'g-', linewidth=2, alpha=0.7, label='Clean signal')
    ax4.plot(t, white_noise, 'b-', alpha=0.5, label='With white noise')
    ax4.plot(t, transient, 'r-', alpha=0.7, label='With transient')
    
    ax4.set_xlabel('Time (s)')
    ax4.set_ylabel('Amplitude')
    ax4.set_title('Different Types of Noise')
    ax4.set_xlim([0, 10])
    ax4.grid(True, alpha=0.3)
    ax4.legend()
    
    # 5. アレイ応答の方向依存性
    ax5 = plt.subplot(2, 3, 5, projection='polar')
    
    # Irregular array
    theta = np.linspace(0, 2*np.pi, 100)
    
    # Array response (direction-dependent)
    response_regular = 1 + 0.1 * np.cos(4 * theta)  # Nearly uniform
    response_irregular = 1 + 0.3 * np.cos(theta) + 0.2 * np.cos(3 * theta)  # Directional
    
    ax5.plot(theta, response_regular, 'b-', linewidth=2, label='Regular array')
    ax5.plot(theta, response_irregular, 'r--', linewidth=2, label='Irregular array')
    ax5.fill_between(theta, 0, response_regular, alpha=0.3)
    ax5.fill_between(theta, 0, response_irregular, alpha=0.2)
    
    ax5.set_ylim([0, 1.5])
    ax5.set_title('Array Response Directionality')
    ax5.grid(True)
    
    # 6. 推奨される処理フロー
    ax6 = plt.subplot(2, 3, 6)
    ax6.axis('off')
    
    # Create flowchart
    steps = [
        "1. Data Acquisition\n(GPS time sync)",
        "2. Preprocessing\n(Detrend, Filter)",
        "3. Quality Check\n(Coherence, SNR)",
        "4. SPAC/FK Analysis\n(Multiple arrays)",
        "5. Dispersion Curve\n(Combine, Smooth)",
        "6. Inversion\n(S-wave model)"
    ]
    
    y_positions = np.linspace(0.9, 0.1, len(steps))
    
    for i, (step, y) in enumerate(zip(steps, y_positions)):
        # Box
        box = plt.Rectangle((0.1, y-0.08), 0.8, 0.12, 
                           fill=True, facecolor='lightblue', 
                           edgecolor='darkblue', linewidth=2)
        ax6.add_patch(box)
        
        # Text
        ax6.text(0.5, y-0.02, step, ha='center', va='center', 
                fontsize=10, weight='bold')
        
        # Arrow
        if i < len(steps) - 1:
            ax6.arrow(0.5, y-0.08, 0, -0.05, head_width=0.05, 
                     head_length=0.02, fc='darkblue', ec='darkblue')
    
    ax6.set_xlim([0, 1])
    ax6.set_ylim([0, 1])
    ax6.set_title('Recommended Processing Flow', fontsize=12, weight='bold')
    
    plt.tight_layout()
    plt.show()

# 実行
visualize_troubleshooting()

## 7. 分散曲線の結合と品質管理

複数のアレイサイズから得られた分散曲線を適切に結合し、品質管理を行う方法を実装します。

In [ ]:
class DispersionCurveMerger:
    """分散曲線の結合と品質管理を行うクラス"""
    
    def __init__(self):
        self.curves = []
        self.array_info = []
        
    def add_curve(self, frequencies, velocities, array_radius, method='SPAC', 
                  coherence=None, quality_scores=None):
        """分散曲線を追加"""
        curve_data = {
            'frequencies': np.array(frequencies),
            'velocities': np.array(velocities),
            'array_radius': array_radius,
            'method': method,
            'coherence': coherence,
            'quality_scores': quality_scores
        }
        
        # 有効周波数範囲を計算
        min_wavelength = 2 * array_radius
        max_wavelength = 10 * array_radius
        
        # 速度の中央値を使用して周波数範囲を推定
        valid_mask = ~np.isnan(velocities)
        if np.any(valid_mask):
            median_velocity = np.median(velocities[valid_mask])
            curve_data['freq_min'] = median_velocity / max_wavelength
            curve_data['freq_max'] = median_velocity / min_wavelength
        else:
            curve_data['freq_min'] = 0
            curve_data['freq_max'] = np.inf
            
        self.curves.append(curve_data)
        self.array_info.append({
            'radius': array_radius,
            'method': method
        })
        
    def merge_curves(self, overlap_weight=2.0, quality_threshold=0.7):
        """複数の分散曲線を結合"""
        all_data = []
        
        for curve in self.curves:
            frequencies = curve['frequencies']
            velocities = curve['velocities']
            
            # 有効範囲内のデータのみ使用
            valid_mask = (~np.isnan(velocities) & 
                         (frequencies >= curve['freq_min']) & 
                         (frequencies <= curve['freq_max']))
            
            # 品質スコアがある場合は考慮
            if curve['quality_scores'] is not None:
                valid_mask &= (curve['quality_scores'] >= quality_threshold)
            
            # 重み付けを計算
            weights = np.ones(np.sum(valid_mask))
            
            # 有効周波数範囲の端での重み減少
            freq_valid = frequencies[valid_mask]
            freq_center = (curve['freq_min'] + curve['freq_max']) / 2
            freq_width = (curve['freq_max'] - curve['freq_min']) / 2
            
            # ガウシアン重み
            weights *= np.exp(-((freq_valid - freq_center) / freq_width)**2 / overlap_weight)
            
            # コヒーレンスによる重み付け
            if curve['coherence'] is not None:
                weights *= curve['coherence'][valid_mask]
            
            # データを追加
            for f, v, w in zip(freq_valid, velocities[valid_mask], weights):
                all_data.append({'freq': f, 'vel': v, 'weight': w, 
                               'array_radius': curve['array_radius']})
        
        # 周波数でソート
        all_data.sort(key=lambda x: x['freq'])
        
        # 重複データの処理
        merged_freq = []
        merged_vel = []
        merged_std = []
        
        # ビンニング
        freq_bins = np.logspace(np.log10(all_data[0]['freq']), 
                               np.log10(all_data[-1]['freq']), 100)
        
        for i in range(len(freq_bins) - 1):
            bin_data = [d for d in all_data 
                       if freq_bins[i] <= d['freq'] < freq_bins[i+1]]
            
            if bin_data:
                # 重み付き平均
                weights = np.array([d['weight'] for d in bin_data])
                velocities = np.array([d['vel'] for d in bin_data])
                
                if np.sum(weights) > 0:
                    avg_freq = np.mean([d['freq'] for d in bin_data])
                    avg_vel = np.sum(weights * velocities) / np.sum(weights)
                    std_vel = np.sqrt(np.sum(weights * (velocities - avg_vel)**2) / 
                                     np.sum(weights))
                    
                    merged_freq.append(avg_freq)
                    merged_vel.append(avg_vel)
                    merged_std.append(std_vel)
        
        return np.array(merged_freq), np.array(merged_vel), np.array(merged_std)
    
    def plot_all_curves(self, show_merged=True):
        """すべての分散曲線をプロット"""
        plt.figure(figsize=(12, 8))
        
        # 個別の曲線
        colors = plt.cm.rainbow(np.linspace(0, 1, len(self.curves)))
        
        for i, curve in enumerate(self.curves):
            valid_mask = ~np.isnan(curve['velocities'])
            
            # 有効範囲を示す
            plt.axvspan(curve['freq_min'], curve['freq_max'], 
                       alpha=0.1, color=colors[i])
            
            # データ点
            plt.scatter(curve['frequencies'][valid_mask], 
                       curve['velocities'][valid_mask],
                       s=50, alpha=0.7, color=colors[i],
                       label=f"{curve['method']} (r={curve['array_radius']}m)")
        
        # 結合された曲線
        if show_merged:
            merged_f, merged_v, merged_std = self.merge_curves()
            plt.plot(merged_f, merged_v, 'k-', linewidth=3, 
                    label='Merged curve', zorder=10)
            plt.fill_between(merged_f, merged_v - merged_std, 
                           merged_v + merged_std,
                           alpha=0.3, color='gray', 
                           label='Uncertainty')
        
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Phase Velocity (m/s)')
        plt.title('Dispersion Curves from Multiple Arrays')
        plt.xscale('log')
        plt.grid(True, alpha=0.3, which='both')
        plt.legend(loc='best')
        plt.tight_layout()
        plt.show()

# 使用例
def demonstrate_curve_merging():
    """分散曲線の結合のデモンストレーション"""
    merger = DispersionCurveMerger()
    
    # Array 1: Large array (r=100m) - good for low frequencies
    f1 = np.logspace(-0.5, 1.0, 30)
    v1 = 200 * f1**0.12
    v1[f1 > 5] = np.nan  # High frequency limit
    coherence1 = np.exp(-f1/3)
    merger.add_curve(f1, v1, 100, 'SPAC', coherence=coherence1)
    
    # Array 2: Medium array (r=50m) - good for mid frequencies  
    f2 = np.logspace(0, 1.5, 40)
    v2 = 210 * f2**0.11
    v2[f2 < 1] = np.nan  # Low frequency limit
    v2[f2 > 15] = np.nan  # High frequency limit
    coherence2 = np.exp(-(f2-5)**2/20)
    merger.add_curve(f2, v2, 50, 'SPAC', coherence=coherence2)
    
    # Array 3: Small array (r=20m) - good for high frequencies
    f3 = np.logspace(0.5, 2.0, 35)
    v3 = 220 * f3**0.10
    v3[f3 < 5] = np.nan  # Low frequency limit
    coherence3 = np.exp(-(f3-15)**2/50)
    merger.add_curve(f3, v3, 20, 'FK', coherence=coherence3)
    
    # Plot all curves and merged result
    merger.plot_all_curves()
    
    # Get merged curve
    merged_f, merged_v, merged_std = merger.merge_curves()
    
    # Quality assessment
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(merged_f, merged_std/merged_v * 100, 'b-', linewidth=2)
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Coefficient of Variation (%)')
    plt.title('Uncertainty Assessment')
    plt.xscale('log')
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    # Data density
    hist, bins = np.histogram(np.log10(merged_f), bins=20)
    bin_centers = (bins[:-1] + bins[1:]) / 2
    plt.bar(10**bin_centers, hist, width=10**(bins[1]-bins[0]), 
            alpha=0.7, edgecolor='black')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Number of Data Points')
    plt.title('Data Density')
    plt.xscale('log')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return merger

# デモンストレーション実行
merger = demonstrate_curve_merging()

## 8. 逆解析の基礎

分散曲線から地下構造（S波速度構造）を推定する逆解析の基本的な実装を示します。

In [ ]:
def visualize_inversion_process():
    """逆解析プロセスの可視化"""
    fig = plt.figure(figsize=(16, 10))
    
    # 1. Forward modeling concept
    ax1 = plt.subplot(2, 3, 1)
    
    # S-wave velocity model
    depths = np.array([0, 5, 10, 20, 40, 80])
    vs = np.array([150, 200, 300, 500, 800])
    
    for i in range(len(vs)):
        ax1.barh(depths[i], vs[i], height=depths[i+1]-depths[i], 
                left=0, alpha=0.7, edgecolor='black')
        ax1.text(vs[i]/2, (depths[i]+depths[i+1])/2, f'Layer {i+1}\nVs={vs[i]} m/s',
                ha='center', va='center', fontsize=9)
    
    ax1.set_xlabel('S-wave Velocity (m/s)')
    ax1.set_ylabel('Depth (m)')
    ax1.set_title('Layered Earth Model')
    ax1.invert_yaxis()
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim([0, 1000])
    
    # 2. Theoretical dispersion curve
    ax2 = plt.subplot(2, 3, 2)
    
    frequencies = np.logspace(-0.5, 1.5, 50)
    # Simplified theoretical curve (実際にはThomson-Haskell法などを使用)
    wavelengths = vs[0] * 2 / frequencies  # 簡易的な計算
    
    # Depth-dependent velocity
    theoretical_vel = np.zeros_like(frequencies)
    for i, f in enumerate(frequencies):
        depth = wavelengths[i] / 3  # 探査深度
        # Find velocity at that depth
        for j in range(len(depths)-1):
            if depths[j] <= depth < depths[j+1]:
                theoretical_vel[i] = vs[j]
                break
        else:
            theoretical_vel[i] = vs[-1]
    
    ax2.plot(frequencies, theoretical_vel, 'b-', linewidth=2, label='Theoretical')
    ax2.scatter(frequencies[::3], theoretical_vel[::3] * (1 + 0.05*np.random.randn(len(frequencies[::3]))), 
               c='red', s=50, alpha=0.7, label='Observed + noise')
    
    ax2.set_xlabel('Frequency (Hz)')
    ax2.set_ylabel('Phase Velocity (m/s)')
    ax2.set_title('Forward Calculation')
    ax2.set_xscale('log')
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    
    # 3. Misfit function
    ax3 = plt.subplot(2, 3, 3)
    
    # Parameter space (simplified to 1D for visualization)
    vs_test = np.linspace(100, 400, 100)
    misfit = np.zeros_like(vs_test)
    
    true_vs = 250
    for i, v in enumerate(vs_test):
        # Simplified misfit calculation
        misfit[i] = np.sum((v/true_vs - 1)**2) + 0.1*np.random.rand()
    
    ax3.plot(vs_test, misfit, 'b-', linewidth=2)
    ax3.axvline(x=true_vs, color='r', linestyle='--', label='True value')
    ax3.scatter([true_vs], [misfit[np.argmin(np.abs(vs_test-true_vs))]], 
               color='red', s=100, zorder=5)
    
    ax3.set_xlabel('Vs of Layer 1 (m/s)')
    ax3.set_ylabel('Misfit')
    ax3.set_title('Objective Function')
    ax3.grid(True, alpha=0.3)
    ax3.legend()
    
    # 4. Optimization algorithms comparison
    ax4 = plt.subplot(2, 3, 4)
    
    iterations = np.arange(50)
    
    # Different convergence patterns
    ga_misfit = 10 * np.exp(-iterations/20) + 0.5 + 0.5*np.random.rand(len(iterations))
    sa_misfit = 10 * np.exp(-iterations/15) + 0.3 + np.exp(-iterations/5)*np.random.rand(len(iterations))
    gradient_misfit = 10 * np.exp(-iterations/5) + 0.1
    
    ax4.plot(iterations, ga_misfit, 'g-', linewidth=2, label='Genetic Algorithm')
    ax4.plot(iterations, sa_misfit, 'r-', linewidth=2, label='Simulated Annealing')
    ax4.plot(iterations, gradient_misfit, 'b-', linewidth=2, label='Gradient Method')
    
    ax4.set_xlabel('Iteration')
    ax4.set_ylabel('Misfit')
    ax4.set_title('Optimization Algorithm Comparison')
    ax4.set_yscale('log')
    ax4.grid(True, alpha=0.3)
    ax4.legend()
    
    # 5. Uncertainty analysis
    ax5 = plt.subplot(2, 3, 5)
    
    # Multiple solutions
    n_models = 50
    depth_array = np.linspace(0, 50, 100)
    
    for i in range(n_models):
        # Perturbed models
        vs_model = np.zeros_like(depth_array)
        for j, d in enumerate(depth_array):
            for k in range(len(depths)-1):
                if depths[k] <= d < depths[k+1]:
                    vs_model[j] = vs[k] * (1 + 0.1*np.random.randn())
                    break
            else:
                vs_model[j] = vs[-1] * (1 + 0.1*np.random.randn())
        
        ax5.plot(vs_model, depth_array, 'gray', alpha=0.2, linewidth=0.5)
    
    # Best model
    vs_best = np.zeros_like(depth_array)
    for j, d in enumerate(depth_array):
        for k in range(len(depths)-1):
            if depths[k] <= d < depths[k+1]:
                vs_best[j] = vs[k]
                break
        else:
            vs_best[j] = vs[-1]
    
    ax5.plot(vs_best, depth_array, 'r-', linewidth=3, label='Best model')
    
    ax5.set_xlabel('S-wave Velocity (m/s)')
    ax5.set_ylabel('Depth (m)')
    ax5.set_title('Model Uncertainty')
    ax5.invert_yaxis()
    ax5.grid(True, alpha=0.3)
    ax5.legend()
    ax5.set_xlim([0, 1000])
    
    # 6. Resolution analysis
    ax6 = plt.subplot(2, 3, 6)
    
    # Sensitivity kernels for different frequencies
    depths_kernel = np.linspace(0, 100, 200)
    
    for f in [1, 2, 5, 10]:
        wavelength = 200 / f  # Approximate
        kernel = np.exp(-3 * depths_kernel / wavelength)
        kernel = kernel / np.max(kernel)
        ax6.plot(kernel, depths_kernel, linewidth=2, label=f'{f} Hz')
    
    ax6.set_xlabel('Sensitivity')
    ax6.set_ylabel('Depth (m)')
    ax6.set_title('Depth Sensitivity Kernels')
    ax6.invert_yaxis()
    ax6.grid(True, alpha=0.3)
    ax6.legend()
    ax6.set_xlim([0, 1.1])
    
    plt.tight_layout()
    plt.show()

# 実行
visualize_inversion_process()

## まとめ

このチュートリアルでは、微動アレイ観測解析の理論から実践までを包括的に学習しました：

1. **基礎理論**
   - 地震波の種類（実体波と表面波）
   - 位相速度と群速度の違い
   - 表面波の分散性と探査深度の関係

2. **解析手法**
   - SPAC法の数学的導出とベッセル関数の役割
   - FK法による方向性を含む解析
   - データ品質管理と前処理の重要性

3. **実践的な実装**
   - 複数アレイサイズからの分散曲線の結合
   - トラブルシューティングと対策
   - 逆解析による地下構造推定

微動解析は、非破壊的に地下構造を推定できる強力な手法ですが、適切な理論的理解と慎重なデータ処理が成功の鍵となります。

# 微動アレイ観測解析チュートリアル

このノートブックでは、微動アレイ観測データの解析手法を実践的に学びます。

## 学習目標
1. 微動観測の基本概念を理解する
2. SPAC法による位相速度の推定方法を習得する
3. 分散曲線から地下構造を推定する流れを理解する

## 必要なライブラリ
まず、必要なライブラリをインポートしましょう。